# Examen final de Desarrollo de Aplicaciones para la Visualización de Datos

Nombre: JORGE

Apellidos: CANDIA SAN JUAN

Tiempo: 2 horas y 30 minutos

## Contexto del ejercicio

El Ayuntamiento de Madrid desea entender en más detalle la situación del problema habitacional en la ciudad de Madrid basada en múltiples fuentes de datos como alquiler vacacional, evolución del precio de alquiler, inflacción y densidad poblacional.

El objetivo del análisis es buscar patrones para entender mejor que está sucediendo, que barrios son los más estresados y que factores son que hacen que el precio de la vivienda esté en máximos. Por lo tanto, se pide:

1. Realizar un análisis descriptivo de los datos con al menos 6 visualizaciones diferentes. (3 Puntos)
2. Realizar un modelo de regresión lineal que explique (mostrar coeficientes) la dinámica del precio de la vivienda en base a los datos de alquiler vacacional a nivel agregado, no por barrio. (2 Puntos)
3. Montar un dashboard con 4 visualizaciones diferentes cómo mínimo. (2 Punto)
4. Concluir todo este análisis haciendo un informe con 2 páginas como mínimo que incluya, preguntas que se van a responder, dashboard resumen que sustente las preguntas y recomendaciones para mejorar la situación habitacional a partir de los resultados obtenidos de los análisis realizados con los datos. (3 Puntos)

## Juego de datos

Para realizar este análisis se provee un juego de datos con las siguientes variables:


| **Variable**           | **Descripción**                                                                                     | **Tipo**                 |
|-------------------------|-----------------------------------------------------------------------------------------------------|--------------------------|
| `neighbourhood_group`   | Grupo de vecindarios o distritos en Madrid.                                                        | Categórica               |
| `date`                 | Fecha de la observación en formato `YYYY-MM-DD`.                                                   | Temporal                 |
| `m2_price`             | Precio promedio por metro cuadrado en euros.                                                       | Cuantitativa continua     |
| `inflation`            | Inflación mensual como porcentaje (incremento en los precios al consumidor).                       | Cuantitativa continua     |
| `HICP`                 | Índice armonizado de precios al consumidor (indicador de inflación en la zona euro).               | Cuantitativa continua     |
| `population_density`   | Densidad de población anual por km² para toda la Comunidad de Madrid.                                                  | Cuantitativa continua     |
| `listings_count`       | Número total de propiedades listadas en Airbnb en un vecindario.                                   | Cuantitativa discreta     |
| `minimum_nights`       | Número mínimo de noches requerido para alquilar una propiedad.                                     | Cuantitativa discreta     |
| `nigth_price`          | Precio promedio por noche de las propiedades en Airbnb.                                            | Cuantitativa continua     |
| `availability_365`     | Número de días al año que una propiedad está disponible para alquilarse.                           | Cuantitativa discreta     |
| `listing_reviews`      | Número promedio de reseñas por propiedad listada.                                                 | Cuantitativa continua     |
| `number_of_reviews`    | Total acumulado de reseñas para todas las propiedades en un vecindario.                            | Cuantitativa discreta     |
| `reviews_per_month`    | Número promedio de reseñas recibidas por una propiedad al mes.                                     | Cuantitativa continua     |
| `hosts_count`          | Número de anfitriones activos en un vecindario.                                                   | Cuantitativa discreta     |
| `Private_room`         | Número de propiedades que ofrecen habitaciones privadas.                                           | Cuantitativa discreta     |
| `Entire_home`          | Número de propiedades que ofrecen viviendas completas.                                             | Cuantitativa discreta     |
| `Hotel_room`           | Número de propiedades categorizadas como habitaciones de hotel.                                    | Cuantitativa discreta     |
| `Shared_room`          | Número de propiedades que ofrecen habitaciones compartidas.                                        | Cuantitativa discreta     |


Ejemplos de preguntas que se pueden realizar a los datos:
1. ¿Cuál es el precio promedio del metro cuadrado (m2_price) por neighbourhood_group y cómo ha cambiado con el tiempo (date)?
2. ¿Existe una correlación entre el precio por metro cuadrado (m2_price) y la densidad de población (population_density)?
3. ¿Qué patrones estacionales se pueden identificar en las reseñas mensuales (reviews_per_month) o en la disponibilidad anual (availability_365)?
4. ¿Qué vecindarios tienen el mayor número de anfitriones (hosts_count) y cómo se relaciona con el precio de la vivienda?
5. ¿Qué vecindarios muestran el mayor crecimiento en precios (m2_price o nigth_price) entre años (year)?
6. ¿Qué factores (e.g., inflation, availability_365, population_density) explican mejor las variaciones en el precio por metro cuadrado (m2_price) o el precio por noche (nigth_price)?

Nota:
- Puedes hacer group_by para sacar variables agregadas por fecha.
- Se valorará la creatividad en las hipótesis, soluciones y limpieza del código y visualizaciones.

## Entrega del exámen

Compartir Url de github con:
- notebook que responde a las preguntas 1, 2 y 3
- PDF respondiendo a la pregunta 4

Puedes entregar el exámen en el siguiente enlace:

https://forms.gle/xe3En2raC3C3UUY39


# Librerías necesarias

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Análisis descriptivos - EJERCICIO 1


In [3]:
df = pd.read_csv("housing_time_series_by_madrid_neighbourhood.csv")

# Exploración inicial
print("Dimensiones del dataset:", df.shape)
print("\nColumnas del dataset:\n", df.columns)
print("\nMuestra de los datos:\n")
print(df.head())
print("\nInformación del dataset:\n")
print(df.info())
print("\nEstadísticas descriptivas:\n")
print(df.describe(include='all'))

# Convertimos 'date' a tipo datetime si no lo está ya
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Extraemos año y mes para análisis temporal
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month


Dimensiones del dataset: (2218, 18)

Columnas del dataset:
 Index(['neighbourhood_group', 'date', 'm2_price', 'inflation', 'HICP',
       'population_density', 'listings_count', 'minimum_nights', 'nigth_price',
       'availability_365', 'listing_reviews', 'number_of_reviews',
       'reviews_per_month', 'hosts_count', 'Private_room', 'Entire_home',
       'Hotel_room', 'Shared_room'],
      dtype='object')

Muestra de los datos:

  neighbourhood_group        date  m2_price  inflation  HICP  \
0              Centro  2010-08-01      15.3        0.3   1.6   
1              Centro  2011-01-01      15.6       -0.7   3.0   
2              Centro  2011-02-01      15.4        0.1   3.4   
3              Centro  2011-05-01      15.0        0.0   3.4   
4              Centro  2011-06-01      15.0       -0.1   3.0   

   population_density  listings_count  minimum_nights  nigth_price  \
0                 796               1             2.0   114.000000   
1                 800               1   

In [4]:
# 1. Evolución temporal del precio promedio del m2 (m2_price) por grupo de vecindarios
fig = px.line(df, x='date', y='m2_price', color='neighbourhood_group', title="Evolución del precio promedio por m² según distrito")
fig.update_layout(xaxis_title="Fecha", yaxis_title="Precio por m² (euros)")
fig.show()



In [5]:
# 2. Distribución del precio por m2 (global)
fig = px.histogram(df, x='m2_price', nbins=30, title="Distribución del precio por m² (global)")
fig.update_layout(xaxis_title="Precio por m² (euros)", yaxis_title="Frecuencia")
fig.show()

In [6]:
# 3. Correlación entre m2_price y population_density
fig = px.scatter(df, x='population_density', y='m2_price', opacity=0.5, title="Relación entre densidad de población y precio por m²")
fig.update_layout(xaxis_title="Densidad de población (personas/km²)", yaxis_title="Precio por m² (euros)")
fig.show()

In [7]:
# 4. Evolución temporal de las reseñas mensuales (reviews_per_month)
monthly_reviews = df.groupby(['year','month'])['reviews_per_month'].mean().reset_index()
monthly_reviews['date_agg'] = pd.to_datetime(monthly_reviews['year'].astype(str) + '-' + monthly_reviews['month'].astype(str) + '-01')
fig = px.line(monthly_reviews, x='date_agg', y='reviews_per_month', title="Evolución temporal de las reseñas mensuales (promedio)")
fig.update_layout(xaxis_title="Fecha", yaxis_title="Reseñas por mes (promedio)")
fig.show()


In [8]:
# 5. Número de anfitriones (hosts_count) por distrito (última fecha disponible)
latest_date = df['date'].max()
latest_data = df[df['date'] == latest_date].groupby('neighbourhood_group', as_index=False)['hosts_count'].mean()
fig = px.bar(latest_data, x='neighbourhood_group', y='hosts_count',
             title="Número promedio de anfitriones por distrito (última fecha disponible)")
fig.update_layout(xaxis_title="Distrito", yaxis_title="Número de anfitriones")
fig.show()

In [9]:
# 6. Cambios en el precio por m² entre años (Crecimiento interanual)
annual_price = df.groupby('year')['m2_price'].mean().reset_index()
annual_price['pct_change'] = annual_price['m2_price'].pct_change() * 100


In [10]:
# Línea de evolución anual del precio
fig = px.line(annual_price, x='year', y='m2_price', markers=True, title="Evolución anual del precio por m² (media global)")
fig.update_layout(xaxis_title="Año", yaxis_title="Precio por m² (euros)")
fig.show()

In [11]:
# Barras del cambio porcentual interanual
fig = px.bar(annual_price.dropna(), x='year', y='pct_change', title="Cambio porcentual interanual del precio por m²")
fig.update_layout(xaxis_title="Año", yaxis_title="Cambio % interanual")
fig.show()

In [12]:
# 1. Evolución temporal del listings count por grupo de vecindarios
fig = px.line(df, x='date', y='listings_count', color='neighbourhood_group', title="Evolución del precio promedio por m² según distrito")
fig.update_layout(xaxis_title="Fecha", yaxis_title="Precio por m² (euros)")
fig.show()

# EJERCICIO 2

In [13]:
# Agrupamos por fecha tomando la media de cada fecha para tener una serie de tiempo agregada
df_agg = df.groupby('date').agg({
    'm2_price': 'mean',
    'listings_count': 'mean',
    'minimum_nights': 'mean',
    'nigth_price': 'mean',
    'availability_365': 'mean',
    'listing_reviews': 'mean',
    'number_of_reviews': 'mean',
    'reviews_per_month': 'mean',
    'hosts_count': 'mean',
    'Private_room': 'mean',
    'Entire_home': 'mean',
    'Hotel_room': 'mean',
    'Shared_room': 'mean'
}).reset_index()

# Eliminamos NAs por si acaso
df_agg = df_agg.dropna()

# Definimos la variable dependiente y las independientes
X = df_agg[['listings_count','minimum_nights',
            'nigth_price','availability_365','listing_reviews','number_of_reviews',
            'reviews_per_month','hosts_count','Private_room','Entire_home','Hotel_room',
            'Shared_room']]
y = df_agg['m2_price']

# Dividimos el conjunto en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creamos y entrenamos el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)

# Coeficientes del modelo
coef_df = pd.DataFrame({
    'variable': X.columns,
    'coeficiente': model.coef_
}).sort_values('coeficiente', ascending=False)

print("\nCoeficientes del modelo de regresión lineal:")
print(coef_df)

# Puntaje del modelo en el set de test
r2_score = model.score(X_test, y_test)
print("\nR² del modelo en el set de prueba:", r2_score)



Coeficientes del modelo de regresión lineal:
             variable  coeficiente
6   reviews_per_month     0.629366
11        Shared_room     0.197955
0      listings_count     0.074609
2         nigth_price     0.021586
3    availability_365     0.005096
7         hosts_count     0.003045
5   number_of_reviews     0.001484
4     listing_reviews    -0.001282
10         Hotel_room    -0.003462
8        Private_room    -0.044824
9         Entire_home    -0.075060
1      minimum_nights    -0.076447

R² del modelo en el set de prueba: 0.8290173443569777


# EJERCICIO 3

In [20]:
# Creamos figura con subplots (2 filas, 2 columnas)
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=("Evolución m² por distrito",
                                    "Distribución precio m²",
                                    "Evolución reseñas mensuales",
                                    "Nº anfitriones último periodo"))

# 1. Evolución del precio promedio por m² según distrito
# Agregamos un gráfico de líneas por cada distrito
for ng in df['neighbourhood_group'].unique():
    data_ng = df[df['neighbourhood_group'] == ng].groupby('date')['m2_price'].mean().reset_index()
    fig.add_trace(
        go.Scatter(x=data_ng['date'], y=data_ng['m2_price'], mode='lines', name=ng),
        row=1, col=1
    )

# 2. Distribución del precio por m² (histograma)
fig.add_trace(
    go.Histogram(x=df['m2_price'], name='Distribución m²'),
    row=1, col=2
)

# 3. Evolución temporal de reseñas mensuales (promedio)
fig.add_trace(
    go.Scatter(x=monthly_reviews['date_agg'], y=monthly_reviews['reviews_per_month'], mode='lines', name='Reseñas/mes'),
    row=2, col=1
)

# 4. Número de anfitriones por distrito (última fecha)
fig.add_trace(
    go.Bar(x=latest_data['neighbourhood_group'], y=latest_data['hosts_count'], name='Anfitriones'),
    row=2, col=2
)

# Ajustamos los títulos ejes y layout
fig.update_xaxes(title_text="Fecha", row=1, col=1)
fig.update_yaxes(title_text="Precio m² (euros)", row=1, col=1)

fig.update_xaxes(title_text="Precio m² (euros)", row=1, col=2)
fig.update_yaxes(title_text="Frecuencia", row=1, col=2)

fig.update_xaxes(title_text="Fecha", row=2, col=1)
fig.update_yaxes(title_text="Reseñas/mes (prom)", row=2, col=1)

fig.update_xaxes(title_text="Distrito", row=2, col=2)
fig.update_yaxes(title_text="Nº Anfitriones", row=2, col=2)

fig.update_layout(title="Dashboard Vivienda Madrid", height=800, width=1200, showlegend=False)

fig.show()